In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import pandas as pd

In [3]:
from tqdm.notebook import tqdm
import sys
import os
import re
from glob import glob
from time import sleep

prefix = os.getcwd().split("jdi-qasp-ml")[0]
sys.path.append(os.path.join(prefix, "jdi-qasp-ml"))

In [4]:
dataset_path = os.path.join(prefix, "jdi-qasp-ml", "data/mui_dataset")

SITE_DFS = [
    p
    for p in glob(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/mui_dataset/df/site*')
]
DF_NAMES = [re.search("site-[0-9]+", nm)[0] for nm in SITE_DFS]

In [5]:
# total_df = pd.DataFrame()
# for i, df_path in tqdm(enumerate(SITE_DFS)):
#     df = pd.read_pickle(df_path)
#     df['site'] = DF_NAMES[i]

#     par_childs = {par:list(df[df.parent_id==par].element_id.unique()) for par in df.parent_id.unique()}

#     def find_all_childs(elem_id):
#         if par_childs.get(elem_id) is None:
#             return []
#         else:
#             res = []
#             for i in par_childs.get(elem_id):
#                 res.append(i)
#                 res.extend(find_all_childs(i))
#             return res
    
#     df['all_followers'] = df['element_id'].apply(find_all_childs)

#     df['STRING'] = df.attributes.apply((lambda x: '' if x is None else x.get('class', "") + " " + x.get('type', "") + " " + x.get('role', "")))
#     df['STRING'] = df.tag_name.str.lower().str.cat(df['STRING'], sep=" ")
#     df['LABEL'] = df.attributes.apply(lambda x: None if x is None else x.get('data-label'))
#     df = df[["site", "element_id", "parent_id", "all_followers", "STRING", "LABEL"]]
#     df.reset_index(drop=True, inplace=True)

#     elem_string_dict = {k:v.get('STRING') for k,v in df[['STRING']].set_index(df.element_id).fillna('').T.to_dict().items()}

#     def concat_all_strings(list):
#         res = ''
#         for l in list:
#             res = res + elem_string_dict.get(l) + ' '
#         return res
    
#     df['all_followers_string'] = df['all_followers'].apply(concat_all_strings)
#     df['final_string'] = (df.STRING + ' ' + df.all_followers_string).fillna('')

#     df = df[["site", "element_id", "parent_id", "final_string", "LABEL"]]
#     total_df = pd.concat([total_df, df])
# total_df.reset_index(inplace=True, drop=True)

In [6]:
# total_df.tail(3)

In [7]:
# total_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model.csv', index=False)

In [8]:
# total_df = pd.read_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model.csv')
# total_df.LABEL = total_df.LABEL.fillna("n/a")


In [9]:
# total_df.tail()

In [10]:
def prepare_string(string):
    string = re.sub(r"(?<=\w)([A-Z])", r" \1", string)
    string = str(string).lower().strip()
    return(re.sub(' +', ' ', re.sub('(-)|([0-9]+)', ' ', string).strip()))


# total_df['final_string'] = total_df['final_string'].apply(prepare_string)

In [11]:
# total_df.tail()

In [12]:
# total_df.to_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model_prepared.csv', index=False)

In [13]:
total_df = pd.read_csv(f'{os.path.join(prefix, "jdi-qasp-ml")}/data/other/total_df_for_nlp_model_prepared.csv')
total_df.LABEL = total_df.LABEL.fillna("n/a")

In [14]:
total_df.tail()

,site,element_id,parent_id,final_string,LABEL
739896,site-13,9627524316559287764950874349,8293726446559287768833831671,div,n/a
739897,site-13,9638773789559287760038756767,8293726446559287768833831671,div,n/a
739898,site-13,4889127663559287761125737880,8293726446559287768833831671,div mui paper root mui popover paper mui paper...,n/a
739899,site-13,8651010676559287765452773916,4889127663559287761125737880,p mui typography root jss mui typography body,typography
739900,site-13,9980562521559287761155767932,8293726446559287768833831671,div,n/a


In [15]:
train_dfs = DF_NAMES[:450]
test_dfs = DF_NAMES[450:]

train = total_df[total_df.site.isin(train_dfs)].copy()
test = total_df[total_df.site.isin(test_dfs)].copy()

In [16]:
train[train.LABEL=="button"]

,site,element_id,parent_id,final_string,LABEL
153,site-36,6012700921559833475778801775,8083030401559833473040253487,button mui button base root mui button root mu...,button
156,site-36,3055060397559833477516072019,8083030401559833473040253487,button mui button base root mui button root mu...,button
239,site-36,1494148025559833470349324986,2047237416559833471527487176,button mui button base root mui button root mu...,button
282,site-36,6202445220559833479221288362,5690271524559833471546038711,div mui button base root mui list item root mu...,button
287,site-36,6234426714559833478997189828,5690271524559833471546038711,div mui button base root mui list item root mu...,button
...,...,...,...,...,...
664196,site-15,1304723266555988633183829078,0537889071555988630964595256,div mui button base root mui list item root mu...,button
664201,site-15,6733012658555988634410515214,0537889071555988630964595256,div mui button base root mui list item root mu...,button
664205,site-15,0289113842555988634893828622,0537889071555988630964595256,div mui button base root mui list item root mu...,button
664210,site-15,6583630390555988633320450650,0537889071555988630964595256,div mui button base root mui list item root mu...,button


Create text pipeline

In [17]:
import itertools
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(prepare_string(text))

vocab = build_vocab_from_iterator(yield_tokens(train.final_string))

In [18]:
print(len(vocab))
print(vocab['accordion'])

277
61


In [19]:
classes_path = os.path.join(prefix, "jdi-qasp-ml", "data/mui_dataset/classes.txt")
with open(classes_path, "r") as f:
    lines = f.readlines()
    classes_dict = {v.strip(): i for i, v in enumerate(lines)}
    classes_reverse_dict = {i: v.strip() for i, v in enumerate(lines)}

train.LABEL = train.LABEL.apply(lambda x: classes_dict.get(x, 0))

In [20]:
text_pipeline = lambda x: vocab(tokenizer(prepare_string(x)))

In [21]:
text_pipeline("div MuiPaper-root MuiPopover-paper MuiPaper")

[7, 0, 38, 1, 0, 170, 38, 0, 38]